# 18. 자바스크립트와 XML
> 다루는 내용
    - 브라우저의 XML DOM 지원 알아보기
    - 자바스크립트 XPath에 대한 이해
    - XSLT 프로세서 사용
    
## 18.1 브라우저의 XML DOM 지원
- DOM 레벨 2
    - XML DOM의 동적생성 도입
- DOM 레벨 3
    - 파싱, 직렬화 추가

### 18.1.1 DOM 레벨 2 코어
- DOM 레벨2에서 document.implementation에 createDocument() 메서드 도입
- `var xmldom = document.implementation.createDocument(namespaceUri, root, doctype)`
- 보통 root 매개변수만 사용

```javascript
var xmldom = document.implementation.createDocument("", "root", null);

console.log(xmldom.documentElement.tagName);

var child = xmldom.createElement("child");
xmldom.documentElement.appendChild(child);
```

- 위 예제는 기본 네임스페이스와 독타입이 없는 XML DOM문서를 생성

---

- 브라우저에서 DOM 레벨 2 XML 을 지원하는지 확인하는 코드
    - `var hasXmlDOm = document.implementation.hasFeature("XML", "2.0");`
- 현실적으로 DOM 메서드만으로 XML 문서를 생성하고 살을 붙여 나가는 일은 드물다
- XML 문서를 파싱해 DOM 구조를 만들거나 그 반대로 진행
- DOM 레벨 2에서 이런 기능을 제공하지 않으므로 브라우저 제조사가 개발한 기능이 사실상의 표준이 됨

### 18.1.2 DOMParser 타입
- 파이어폭스가 도입한 DOMParser 타입은 XML을 파싱하여 DOM 문서를 만드는데 특화
- 이후 IE9, 사파리, 크롬, 오페라에서 수용
- 인스턴스를 생성하 다음 `parseFromString()` 호출
    - 매개변수로 파싱할 XML 문자열과 콘텐츠타입을 받는데, 콘텐츠 타입은 항상 "text/xml"
    - 매서드 반환 값은 document의 인스턴스
    
```javascript
var parser = new DOMParser();
var xmldom = parser.parseFromString('<root><child/></root>', 'text/xml');

console.log(xmldom.documentElement.tagName);
console.log(xmldom.documentElement.firstChild.tagName);

var anotherChild = xmldom.createElement('child');
xmldom.documentElement.appendChild(anotherChild);

var children = xmldom.getElementsByTagName('child');
console.log(children.length);
```

---
- DOMParser는 형식에 맞는 XML만 파싱할 수 있으므로 HTML을 파싱해 HTML문서를 만들수는 없다.
- 파싱 에러가 발생했을 때의 반응은 브라우저마다 다르다
- 파이어폭스, 오페라, 사파리, 크롬에서는 파싱에러가 발생해도 Document 객체를 반환하지만 이 객체의 document요소는 `<parsererror>` 이며 콘텐츠는 파싱 에러에 대한 설명이다.
- 파싱 에러가 일어났는지 판단하는 최선의 방법은 다음과 같이 try-catch문을 쓰고, 에러가 없다면 `getElementByTagName()`로 `<parsererror>` 요소를 확인하는 것이다

```javascript
var parser = new DOMParser(), xmldom, errors;
try {
    xmldom = parser.parseFromString('<root>', 'text/xml');
    errors = xmldom.getElementsByTagName('parsererror');
    if (errors.length > 0) {
        throw new Error('Parsing error!');
    }
} catch (ex) {
    alert('Parsing error!');
}
```
- `</root>` 태그가 없으므로 파싱 에러 발생

### 18.1.3 XMLSerializer 타입
- 파이어폭스는 DOMParser 의 짝으로 XMLSerializer 타입도 도입
- DOM문서를 직렬화하여 XML 문자열로 만드는 것

```javascript
var serializer = new XMLSerializer();
var xml = serializer.serializeToString(xmldom);
console.log(xml);
```

- serializeToString()이 반환하는 문자열은 사람이 눈으로 검토할 것을 고려하고 만들어지지 않았음
- XMLSerializer은 유효한 DOM객체라면 무엇이든 직렬화할 수 있으며, 각 노드, HTML문서도 포함

### 18.1.4 인터넷 익스플로러 8 및 이전의 XML
- 네이티브 XML 처리를 처음 구현한 브라우저는 인터넷익스플로러
- Active X 객체를 통해 네이티브 XML 처리 지원
- 마이크로 소프트에서는 데스크톱 어플리케이션 개발자에게 XML 처리 기능을 제공할 MSXML 라이브러리를 만들었는데, 자바스크립트에서 사용할 라이브러리를 따로 만들지 않고 브라우저에서도 MSXML 라이브러리에 접근할 수 있게 했음
- 자세한 내용은 생략

### 18.2 브라우저의 XPath(XML Path Language) 지원
- XPath는 DOM 문서에서 특정 노드를 찾아내는 방법
- XML 문서의 특정 요소나 속성에 접근하기 위한 경로를 지정하는 언어
- [xml_xpath](http://tcpschool.com/xml/xml_xpath_intro)

### 18.2.1 DOM 레벨 3 XPath
- 브라우저가 DOM 레벨 3 XPath를 지원하는지 확인
    - `var supportsXPath = document.implementation.hasFeature('XPath', '3.0');`
- 명세에는 여러 가지 타입이 정의되어 있는데 그중 가장 중요한 것은 XPathEvaluator와 XPathResult
- XPathEvaluator는 특정 컨텍스트 내에서 XPath 표현식을 평가한다.
    - 다음과 같은 메서드가 들어 있다.
        - createExpression(expression, nsresolver)
            - XPath 표현식과 네임스페이스 정보를 조합해서 컴파일된 쿼리인 XPathExpresiion을 만든다.
            - 같은 쿼리를 여러번 반복할 때 유용하다
        - createNSResolver(node)
            - node의 네임스페이스 정보를 바탕으로 새 XPathNSResolver 객체를 생성
            - 네임스페이스를 사용하는 XML 문서에 대해 평가할 때는 XPathNSResolver 객체가 꼭 필요하다
        - evaluate(expression, context, nsresolver, type, result)
            - 주어진 컨텍스트와 네임스페이스 정보를 바탕으로 XPath 표현식을 평가한다.
            - 가장 자주 쓰이는 메서드
            
```javascript
var result = xmldom.evaluate(
    'employee/name', xmldom.documentElement, null, XPathResult.ORDERED_NODE_ITERATOR_TYPE, null);

if (result !== null) {
    var element = result.iterateNext();
    while (element) {
        console.log(element.tagName);
        node = result.iterateNext();
    }
}
```

## 18.3 브라우저의 XSLT 지원
- XSLT는 XML과 짝을 이루는 기술이며 XPath를 이용해 문서 표현을 다른 형식으로 바꾼다
- [xml_xslt](http://tcpschool.com/xml/xml_xslt_intro)

### 18.3.1 인터넷 익스플로러의 XSLT
pass

### 18.3.2 XSLTProcessor 타입
- 첫번째 단계
    - XML DOM 문서와 XSLT DOM 문서를 불러오는 것
    - 새 XSLTProcessor를 생성하고, importStylesheet() 메서드를 써서 XSLT를 할당한다.
    
```javascript
// XsltProcessorExample01.htm
var processor = new XSLTProcessor();
processor.importStylesheet(xsltdom);
```

- 마지막 단계
    - 실제 변형
    - 변형 방법은 두 가지이다.
        - 완전한 DOM 문서를 반환하려면 transformToDocument()를 호출
        - transformToFragment()를 써서 문서 버퍼 객체로 반환할 수도 있다.
        



                                        